In [1]:
import pandas as pd
import caveclient
import numpy as np
import nglui

In [3]:
url = "https://global.brain-wire-test.org/"
datastack = "h01_c3_flat"
token = "3ac8d9f0de1b192dba9c085114f0f811"

In [6]:
client = caveclient.CAVEclient(datastack_name=datastack, server_address=url, auth_token=token)

In [7]:
df = pd.read_hdf('../data/dbcells_dump_table_data.h5')

In [8]:
df.head()

,valid,classification_system,cell_type,pt_position
0,True,"0, 0, 0",UNKNOWN0,"[0, 0, 0]"
1,True,"0, 0, 0",UNKNOWN0,"[0, 0, 0]"
2,True,"0, 0, 0",UNKNOWN0,"[0, 0, 0]"
3,True,"0, 0, 0",UNKNOWN0,"[0, 0, 0]"
4,True,"0, 0, 0",UNKNOWN0,"[0, 0, 0]"


In [10]:
client.info.image_source()

'precomputed://gs://h01-release/data/20210601/4nm_raw'

In [9]:
img_layer = nglui.statebuilder.ImageLayerConfig(client.info.image_source(), name='img')


In [11]:
pt_mapper = nglui.statebuilder.PointMapper(point_column='pt_position')
ann_layer = nglui.statebuilder.AnnotationLayerConfig('anno', mapping_rules=[pt_mapper])


In [12]:
sb = nglui.statebuilder.StateBuilder([img_layer, ann_layer], url_prefix='https://ngl.cave-explorer.org')


In [28]:
d=sb.render_state(df.iloc[0:20], return_as='dict', url_prefix='https://ngl.cave-explorer.org',
                compatibility_mode=True)

In [29]:
d['layers'][1]['source'][0]['transform']['outputDimensions']=    {
            "x": [
              8e-9,
              "m"
            ],
            "y": [
              8e-9,
              "m"
            ],
            "z": [
              3.3e-8,
              "m"
            ]
          }

In [30]:
d['crossSectionScale']=2
d['projectionScale']=30000


In [31]:
#d['layers'][1].pop('linkedSegmentationLayer')

In [32]:
state_id=client.state.upload_state_json(d)

In [33]:
state_id

5146660971741184

In [34]:
url=client.state.build_neuroglancer_url(state_id, ngl_url = 'http://ngl.cave-explorer.org/').replace("/?json_url=", "#!middleauth+")

In [35]:
url

'http://ngl.cave-explorer.org#!middleauth+https://global.brain-wire-test.org//nglstate/api/v1/5146660971741184'